In [73]:
import pandas as pd
from jaal import Jaal
from jaal.datasets import load_got
import matplotlib.pyplot as plt

In [74]:
sz_file_path = "./dump.csv"
sz_file_csv = pd.read_csv(sz_file_path,usecols=['initiatingDeviceName','receivingDeviceName', 'connectionCount'],
                          header=0,
                       
                        )

In [75]:
df = sz_file_csv.rename(
    columns={
        "initiatingDeviceName": "from",
        "receivingDeviceName": "to",
        "connectionCount": "weight"
    }
)

In [76]:
df_grouped = df.groupby(['from','to'], as_index=False)['weight'].sum()

In [77]:
df = sz_file_csv.rename(
    columns={
        "initiatingDeviceName": "id",
        "receivingDeviceName": "to",
        "connectionCount": "weight",

    }
)
df_nodes = df.groupby(['id'], as_index=False)['weight'].sum()

In [80]:
#Jaal(df_grouped).plot(port=8071)
Jaal(df_grouped,df_nodes).plot(vis_opts={'height': '600px', # change height
#                                     'interaction':{'hover': True}, # turn on-off the hover                                    
                                    'physics':{'enabled': False}})                     # Estatico





KeyError: "None of [Index(['initiatingDeviceName', 'receivingDeviceName', 'protocol',\n       'connectionCount'],\n      dtype='object')] are in the [columns]"